In [ ]:
#Importamos librerias de utilidad
import numpy as np 
import pylab as pl
import random as rd
import copy

class TrafficSimulation():
    """
        Simulación de tráfico de carril N basada en Rickert et al. (1996)
        y el modelo original de Nagel & Schreckenberg (1992).

    """

    def __init__(self, len_road = 100, density = 0.01, max_vel = 5, 
                 prob_slowing = 0.5, prob_lane_change = 0.8, lanes = 1):
        self.len_road = len_road
        self.density = density
        self.max_vel = max_vel
        self.prob_slowing = prob_slowing
        self.prob_lane_change = prob_lane_change
        self.lanes = lanes

        self.car_counter = 0
        self.flow_counter = 0
        self.time_counter = 0

        self.config = []
        self.next_config = []


    def initialize(self):
        """
        Establece el estado inicial en función de la densidad y la longitud e
        inicia el conteo global de autos.
        """
        for _ in range(self.lanes):
            self.config.append(pl.zeros([self.len_road], dtype=int))
            self.next_config.append(pl.zeros([self.len_road], dtype=int))

        for lane in self.config:
            lane.fill(-1)
        for lane in self.next_config:
            lane.fill(-1)

        for state in self.config:
            indexes = np.random.choice(range(0, self.len_road),
                      size = int(self.density * self.len_road),
                      replace = False)
            state[indexes] = self.max_vel

        self.car_counter = self.count_cars()


    def lane_change(self):
        """
        Comprueba los cambios de carril y mueve los carros entre carriles.
        Rules:
        IF  v > gap
        AND v_o + 1 are empty AND empty behind >= vmax
        AND IF rd.rand < probability of switching
        THEN switch.
        """
        for lane_index, lane in enumerate(self.config):
            for spot in range(self.len_road):
                if lane[spot] != -1:                        #if car
                    switch = False                          #if car will switch lane
                    vel = lane[spot]

                    gap = 0                                 #rule 1 - check ahead            
                    looked_ahead = 0

                    while looked_ahead <= (self.max_vel):
                        if lane[(spot + gap + 1) % self.len_road] < 0:
                            gap += 1
                        else:
                            break
                        looked_ahead += 1

                    if gap < vel:                           #rule 1 - if no space ahead, switch
                        switch = True

                                                            #rule 2 - define neighbors
                    right_neighbor = self.config[(lane_index + 1) % self.lanes]
                    left_neighbor = self.config[(lane_index - 1) % self.lanes]

                    if switch:                              #rule 2 - if planning on switching
                        switch_right = False                
                        switch_left = False
                                                            #rule 2 - check for free lanes
                        if self.neighbor_free(right_neighbor, spot, vel):
                            switch_right = True
                        if self.neighbor_free(left_neighbor, spot, vel):
                            switch_left = True

                                                            #rule 3 - if less than random switching 
                                                            #probability, switch lanes
                        if rd.random() < self.prob_lane_change:
                            self.switch_lanes(lane_index, spot, vel, switch_right, switch_left)
                        else:
                            self.next_config[lane_index][spot] = vel

                                                            #if we're not switching the car, then we
                    else:                                   #have to write it back into it's own lane
                        self.next_config[lane_index][spot] = vel

        # self.state_difference()                                                    
        self.config = copy.deepcopy(self.next_config)        #update config and resetnext_config state
        for lane in self.next_config:                        
            lane.fill(-1)
